In [1]:
import sqlite3

In [2]:
# Connexion à la base de données SQLite
conn = sqlite3.connect('examp5.db')
cursor = conn.cursor()

# Création de la table
cursor.execute('''
CREATE TABLE IF NOT EXISTS produits (
    id INTEGER PRIMARY KEY,
    nom TEXT,
    stock INTEGER
)
''')

# Initialisation des données de test
cursor.execute("INSERT INTO produits (nom, stock) VALUES ('Produit A', 10)")
cursor.execute("INSERT INTO produits (nom, stock) VALUES ('Produit B', 20)")
conn.commit()

In [3]:
def transaction_avec_points_de_restauration():
    try:
        cursor.execute("BEGIN")
        print("Début de la transaction")

        # Point de restauration initial
        cursor.execute("SAVEPOINT point1")
        cursor.execute("UPDATE produits SET stock = stock - 5 WHERE nom = 'Produit A'")
        print("Déstockage de 5 unités pour Produit A")

        # Point de restauration secondaire
        cursor.execute("SAVEPOINT point2")
        cursor.execute("UPDATE produits SET stock = stock - 5 WHERE nom = 'Produit B'")
        print("Déstockage de 5 unités pour Produit B")

        # Simuler une erreur et retour au point de restauration initial
        print("Simuler une erreur")
        raise Exception("Erreur simulée")

        # Point de restauration tertiaire
        cursor.execute("SAVEPOINT point3")
        cursor.execute("UPDATE produits SET stock = stock + 25 WHERE nom = 'Produit B'")
        print("Déstockage de 5 unités pour Produit B")
        
        cursor.execute("RELEASE SAVEPOINT point3")
        cursor.execute("RELEASE SAVEPOINT point2")
        cursor.execute("RELEASE SAVEPOINT point1")
        conn.commit()
        print("Transaction validée")

    except Exception as e:
        cursor.execute("ROLLBACK TO point2")
        print(f"Transaction échouée: {e}")

# Exécution de la fonction
transaction_avec_points_de_restauration()

Début de la transaction
Déstockage de 5 unités pour Produit A
Déstockage de 5 unités pour Produit B
Simuler une erreur
Transaction échouée: Erreur simulée


In [4]:
# Vérification de l'état final des stocks
cursor.execute("SELECT * FROM produits")
produits = cursor.fetchall()
for produit in produits:
    print(f"Produit {produit[1]}: Stock = {produit[2]}")

conn.close()

Produit Produit A: Stock = 5
Produit Produit B: Stock = 20
Produit Produit A: Stock = 5
Produit Produit B: Stock = 20
Produit Produit A: Stock = 5
Produit Produit B: Stock = 20


#### Interprétation des résultats
1. **Début de la transaction** :
   - La transaction commence, et un point de restauration initial (`point1`) est défini.
   
2. **Premier point de restauration** :
   - Le stock du `Produit A` est décrémenté de 5 unités et un deuxième point de restauration (`point2`) est créé.

3. **Erreur simulée** :
   - Une erreur est volontairement générée, simulant un problème dans la transaction.

4. **Retour au point de restauration initial** :
   - En raison de l'erreur, nous revenons au `point2`, annulant toutes les opérations après ce point.
   - Le `RELEASE SAVEPOINT` permet de libérer les points de restauration qui ne sont plus nécessaires.

5. **Résultat final** :
   - Seules les opérations avant le point de restauration initial (`point2`) sont validées.
   - L'état final des stocks montre que seul le déstockage de `Produit A` a eu lieu.